In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('/srv/home/christinedk/wp_internship/collaboration/')
from utils import get_edits_pre_tag
from features.article_history import *
from features.politeness import *

/usr/lib/python3/dist-packages/sklearn/feature_extraction/image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):
/usr/lib/python3/dist-packages/sklearn/linear_model/least_angle.py:35: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.

In [3]:
import pandas as pd
pd.options.mode.chained_assignment = None
from dateutil import parser
import datetime
import numpy as np
from tqdm import tqdm
import json
import matplotlib.pylab as plt
from math import log2
from features.talk_history import ConvParser

In [4]:
# chained

In [5]:
HOME = '/srv/home/christinedk/wp_internship/'
DATA_DIR = HOME + 'data/'

def read_revisions(filename, rename=False):
    revisions = pd.read_json(filename,lines=True)
    if rename:
        revisions = revisions.rename(columns = {'revision_timestamp':'event_timestamp','user_id':'event_user_id'})
    revisions['event_timestamp'] = pd.to_datetime(revisions['event_timestamp'])
    revisions = revisions.sort_values(by='event_timestamp', ascending=True)
    return revisions

def np_encoder(object):
    if isinstance(object, np.generic):
        return object.item()

In [6]:
ls -lt /srv/home/christinedk/wp_internship/data/page_history/

total 3599172
-rw-r--r-- 1 christinedk wikidev  251029235 Mar 23 13:27 page_history-fanpov-meta-info.json
-rw-r--r-- 1 christinedk wikidev 1054296428 Mar 23 13:27 page_history-advert-meta-info.json
-rw-r--r-- 1 christinedk wikidev  338398048 Mar 23 13:26 page_history-autobiography-meta-info.json
-rw-r--r-- 1 christinedk wikidev 1087703473 Mar 23 13:26 page_history-peacock-meta-info.json
-rw-r--r-- 1 christinedk wikidev  954105044 Mar 23 13:25 page_history-weasel-meta-info.json


In [9]:
for template in ['fanpov','weasel','autobiography','advert','peacock']:
    print(template)
    
    print('reading data')
    # read and format things
    page_revisions = read_revisions(DATA_DIR+'page_history/page_history-{}-meta-info.json'.format(template))
    article_talk_mappings = pd.read_csv(DATA_DIR+'article_talk_mappings/{}.csv'.format(template),
                                       usecols=['talk_page_id','article_page_id'])
    labels = pd.read_csv(DATA_DIR+'labels/{}.csv'.format(template),parse_dates=['event_timestamp'])
    labels = labels.join(article_talk_mappings.set_index('article_page_id')[['talk_page_id']],on='page_id')
    
    # prepare to extract by page
    pages = page_revisions.groupby('page_id')
    
    features = []
    for tag_date, page_id, talk_page_id in tqdm(labels.values):
        user_article_feat = article_feat = {}

        page_revisions = pages.get_group(page_id)
        page_revisions = page_revisions[page_revisions.event_timestamp <= tag_date]

        if len(page_revisions) > 0:
            page_revisions = calculate_page_metrics(page_revisions)
            # user-article
            user_article_feat = get_user_article_features(page_revisions)
            # article
            article_feat = get_article_features(page_revisions, tag_date)
                
        features.append({'page':page_id,'date':str(tag_date),
                        'user_article':user_article_feat,
                        'article':article_feat})
    with open(HOME +'features/activity_'+template+'.json','w') as f:
        json.dump(features,f,default=np_encoder)


fanpov
reading data


100%|██████████| 721/721 [00:13<00:00, 51.57it/s]


weasel
reading data


100%|██████████| 1322/1322 [00:26<00:00, 49.85it/s]


autobiography
reading data


100%|██████████| 4224/4224 [01:18<00:00, 53.89it/s]


advert
reading data


100%|██████████| 7570/7570 [02:23<00:00, 52.80it/s]


peacock
reading data


100%|██████████| 5174/5174 [01:38<00:00, 52.45it/s]
